# Inference Notebook
This jupyter notebook is used to get predictions from our trained model for images from test set

1. Retrieve predictions from inference model
2. (Opt.) Visualize predictions
3. Merge individual json files with annotations into the one follows COCO format

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

In [2]:
import os
import sys
from os import walk
from tqdm import tqdm

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  # To find local version of the library

from utils.utils import *
from utils.dataset import ChronsiteTestDataset
from utils.configs import ChronsiteConfig

import mrcnn.model as modellib

Using TensorFlow backend.


In [3]:
# Define working directories

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Directory of images to run detection on
INFERENCE_DIR = os.path.join(ROOT_DIR, "test_images")
assert os.path.exists(INFERENCE_DIR), "You must create this directory manually and upload test images within"
    
# Checkpoint of trained model to reload
MODEL_BRANCH = "chronsite20201003T1222"
CHECKPOINT_EPOCH = 77

# Directory to store all individual json files
INFERENCE_JSON_DIR = os.path.join(INFERENCE_DIR, "annotation")
if not os.path.exists(INFERENCE_JSON_DIR):
    os.mkdir(INFERENCE_JSON_DIR)
    
# Directory to store all visualizations of predictions
INFERENCE_RENDER_DIR = os.path.join(INFERENCE_DIR, "render")
if not os.path.exists(INFERENCE_RENDER_DIR):
    os.mkdir(INFERENCE_RENDER_DIR)
    
# Final path of json file in coco format
FINAL_JSON_PATH = './annotations_group8.json'

# 1. Retrieve predictions from inference model

<font color='red'> Test set was distributed via direct file transfer.</font> </br>
<font color='red'> You must upload images to INFERENCE_DIR first.</font>

In [4]:
# Get the list of samples in the test set,
# since the test set was retrieved by direct file transfer.
sample_ids = []
for (dirpath, dirnames, filenames) in walk(INFERENCE_DIR):
    if dirpath == INFERENCE_DIR:
        for filename in filenames:
            sample_ids.append(filename[:-4])
print(f"Number of samples in test set: {len(sample_ids)}")

Number of samples in test set: 100


In [5]:
# Create dataset object for test set
dataset_inference = ChronsiteTestDataset(INFERENCE_DIR, sample_ids)
dataset_inference.load_kernel()

100%|██████████| 100/100 [00:02<00:00, 47.35it/s]


In [6]:
# Reset config for model in inference mode
config = ChronsiteConfig()
config.IMAGE_RESIZE_MODE = "pad64"
config.IMAGE_MIN_DIM = None
config.IMAGE_MIN_SCALE = None

In [7]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=config,
                          model_dir=MODEL_DIR)



model_path = os.path.join(MODEL_DIR, MODEL_BRANCH,\
                          f"mask_rcnn_chronsite_{CHECKPOINT_EPOCH:04d}.h5")

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  /home/jovyan/eleven/test_w_submodule/maskrcnn/logs/chronsite20201003T1222/mask_rcnn_chronsite_0077.h5






Re-starting from epoch 77


In [8]:
# Retrieve predictions and stored in individual json files
for image_id in tqdm(dataset_inference.image_ids):
    retrieve_inference_to_json(model, dataset_inference, image_id, INFERENCE_JSON_DIR)

100%|██████████| 100/100 [04:42<00:00,  2.82s/it]


# 2(Opt.) Visualize predictions

You can find the rendered and saved pictures in INFERENCE_RENDER_DIR

In [9]:
for image_id in tqdm(dataset_inference.image_ids):
    sample_id = dataset_inference.image_info[image_id]['id']
    visualize_inference_sample(sample_id, dataset_inference.class_names_preset,
                              INFERENCE_DIR, INFERENCE_JSON_DIR, INFERENCE_RENDER_DIR)

100%|██████████| 100/100 [00:36<00:00,  2.72it/s]


# 3. Merge individual json files with annotations into the one follows COCO format

In [10]:
dump_coco_format(sample_ids, INFERENCE_JSON_DIR,
                 FINAL_JSON_PATH, dataset_inference.class_names_preset)

In [11]:
# New json in coco format will be verified if there is no exception or assert triggered.
verify_coco_json(FINAL_JSON_PATH)

CALSSES 5 classes:
{0: 'BG', 1: 'Concrete_pump_hose', 2: 'Mixer_truck', 3: 'People', 4: 'Vertical_formwork'}
 
INFO: {'description': 'Chronsite test set, Group 8.', 'year': 2020}
 
LICENSES: []
 
IMAGES
Shapes: {(1080, 1920), (720, 1280), (1024, 1280)}
Filenames: ['img/frame527.jpg', 'img/2020-07-15-11-49-12.jpg', 'img/frame516.jpg', 'img/2020-07-15-11-56-11.jpg', 'img/frame528.jpg', 'img/frame1628.jpg', 'img/frame521.jpg', 'img/2019-10-17-14-06-24.jpg', 'img/frame510.jpg', 'img/frame0519.jpg', 'img/2019-10-17-14-30-46.jpg', 'img/2019-10-17-14-42-56.jpg', 'img/frame1620.jpg', 'img/2020-07-27-09-54-07.jpg', 'img/frame0509.jpg', 'img/2020-08-13-19-13-58.jpg', 'img/2019-10-17-13-42-19.jpg', 'img/frame515.jpg', 'img/2019-10-17-14-55-14.jpg', 'img/frame0489.jpg', 'img/frame0504.jpg', 'img/2020-07-15-11-36-02.jpg', 'img/frame968.jpg', 'img/2020-07-15-12-01-36.jpg', 'img/2020-07-27-13-29-02.jpg', 'img/frame524.jpg', 'img/frame1624.jpg', 'img/frame0464.jpg', 'img/frame0479.jpg', 'img/2020-07-1